### Model Training

In [1]:
import csv
import pandas as pd

In [2]:
try:
    import models
except ImportError:
    print('Please upload models.py to you current directory')
    raise

2024-04-22 03:43:19.399907: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 03:43:19.450319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 03:43:20.212040: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_pickle("./data/cve")

In [4]:
train = df[:120000]
test = df[120000:]

In [5]:
from models import BERTmodel

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100 80GB PCIe


In [7]:
metric = 'scope'

In [10]:
model = BERTmodel(metric, train, test)

In [12]:
model.train(epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i


======== Epoch 1 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:17:41.
  Batch 4,000  of  13,500.    Elapsed: 0:37:50.
  Batch 6,000  of  13,500.    Elapsed: 0:57:59.
  Batch 8,000  of  13,500.    Elapsed: 1:18:05.
  Batch 10,000  of  13,500.    Elapsed: 1:38:12.
  Batch 12,000  of  13,500.    Elapsed: 1:58:18.

  Average training loss: 0.16
  Training epoch took: 2:13:22

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.16
  Validation took: 0:04:59

======== Epoch 2 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:19:58.
  Batch 4,000  of  13,500.    Elapsed: 0:40:07.
  Batch 6,000  of  13,500.    Elapsed: 1:00:15.
  Batch 8,000  of  13,500.    Elapsed: 1:20:24.
  Batch 10,000  of  13,500.    Elapsed: 1:40:32.
  Batch 12,000  of  13,500.    Elapsed: 2:00:40.

  Average training loss: 0.13
  Training epoch took: 2:15:47

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.14
  Validation took: 0:04:56

Training complete!
Total tr

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.16,0.16,0.97,2:13:22,0:04:59
2,0.13,0.14,0.97,2:15:47,0:04:56


In [13]:
model.test()

0.9446197991391678

In [14]:
print(model.getClassificationStats())

{'accuracy': 0.9446197991391678, 'precision': 0.9446197991391678, 'recall': 0.9446197991391678, 'f1': 0.9446197991391678, 'mcc': 0.8361938788353562, 'cm': array([[23867,   492],
       [ 1245,  5761]])}


In [15]:
model.saveModel()
model.savePredictions()

### Push model to huggingface hub

In [13]:
from huggingface_hub import notebook_login

In [14]:
notebook_login()

In [15]:
from transformers import BertForSequenceClassification

In [16]:
model = BertForSequenceClassification.from_pretrained(f"./{metric}_model/")

In [ ]:
model.push_to_hub(f"Vijaybhk/{metric.title()}-BERT")